<a href="https://colab.research.google.com/github/Arijit-crypt/Portfolio/blob/main/SMS_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
!pip install tensorflow-datasets
!pip install protobuf==3.19.6
!pip install tensorboard-data-server==0.6.1
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.11.0


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-03-05 02:53:28--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  1.67MB/s    in 0.2s    

2023-03-05 02:53:28 (1.67 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2023-03-05 02:53:28--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [3]:
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

In [4]:
label_dict = {'ham':0, 'spam':1}
train_df['label'] = train_df['label'].map(label_dict)
test_df['label'] = test_df['label'].map(label_dict)

In [5]:
vocab = []
for i, row in train_df.iterrows():
  vocab.append(row['message'].split())

In [6]:
vocab = [word for line in vocab for word in line]
VOCAB_SIZE = len(set(vocab))

In [7]:
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_df['message'].values)

In [8]:
model = Sequential(encoder)
model.add(Embedding(input_dim=len(encoder.get_vocabulary()), output_dim=64, mask_zero=True))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 64)          523584    
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 560,833
Trainable params: 560,833
Non-trainable params: 0
__________________________________________________

In [10]:
history = model.fit(x=train_df['message'].values, y=train_df['label'].values, epochs=40, validation_split=0.2)

Epoch 1/40
105/105 [==============================] - 31s 163ms/step - loss: 0.2771 - accuracy: 0.9138 - val_loss: 0.0588 - val_accuracy: 0.9904
Epoch 2/40
105/105 [==============================] - 9s 88ms/step - loss: 0.0297 - accuracy: 0.9937 - val_loss: 0.0697 - val_accuracy: 0.9856
Epoch 3/40
105/105 [==============================] - 5s 52ms/step - loss: 0.0085 - accuracy: 0.9991 - val_loss: 0.0661 - val_accuracy: 0.9833
Epoch 4/40
105/105 [==============================] - 3s 32ms/step - loss: 0.0058 - accuracy: 0.9982 - val_loss: 0.0558 - val_accuracy: 0.9833
Epoch 5/40
105/105 [==============================] - 3s 31ms/step - loss: 0.0018 - accuracy: 0.9997 - val_loss: 0.0652 - val_accuracy: 0.9868
Epoch 6/40
105/105 [==============================] - 4s 38ms/step - loss: 6.7931e-04 - accuracy: 1.0000 - val_loss: 0.0712 - val_accuracy: 0.9821
Epoch 7/40
105/105 [==============================] - 2s 20ms/step - loss: 4.8602e-04 - accuracy: 1.0000 - val_loss: 0.0747 - val_accura

In [11]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = list(model.predict([pred_text])[0])
  if prediction[0] < 0.5:
    prediction.append('ham')
  else:
    prediction.append('spam')
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 2s 2s/step
[2.2831563e-09, 'ham']


In [12]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 49ms/step
You passed the challenge. Great job!
